In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout,
                                     GRU, Reshape, BatchNormalization)
import numpy as np
import os
import cv2

# Load Dataset
data_dir = "D:/Traffic Signal Analysis/archive/Train"  # Change this to your dataset folder
image_size = (32, 32)
num_classes = 43

def load_data(data_dir):
    data = []
    labels = []
    for class_id in range(num_classes):
        class_folder = os.path.join(data_dir, str(class_id))
        for img_name in os.listdir(class_folder):
            img_path = os.path.join(class_folder, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, image_size)
            data.append(img)
            labels.append(class_id)
    return np.array(data), np.array(labels)

X, y = load_data(data_dir)

# Normalize images
X = X / 255.0

# Convert labels to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes)

# Split into train and validation
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define CNN + GRU Model
input_shape = (32, 32, 3)
inputs = Input(shape=input_shape)

# CNN Feature Extractor
x = Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2,2))(x)

x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2,2))(x)

x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2,2))(x)

# Reshape CNN Features for GRU
x = Reshape((x.shape[1] * x.shape[2], x.shape[3]))(x)

# GRU Layer
x = GRU(128, return_sequences=False)(x)

# Fully Connected Layers
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(num_classes, activation='softmax')(x)

# Create Model
model = Model(inputs, x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary
model.summary()

# Train Model
epochs = 20
batch_size = 32
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size)

# Save Model
model.save("traffic_signs_cnn_gru.h5")

# Evaluate on Validation Set
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 32, 32, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 32, 32, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 16, 16, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 16, 16, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 8, 8, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 8, 8, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 16, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 128)                 │          99,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 43)                  │           5,547 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 215,275 (840.92 KB)

 Trainable params: 214,827 (839.17 KB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 30s 26ms/step - accuracy: 0.4031 - loss: 2.1243 - val_accuracy: 0.9109 - val_loss: 0.2942
Epoch 2/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 25s 25ms/step - accuracy: 0.9412 - loss: 0.2036 - val_accuracy: 0.9699 - val_loss: 0.1025
Epoch 3/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 25s 25ms/step - accuracy: 0.9825 - loss: 0.0625 - val_accuracy: 0.9746 - val_loss: 0.0836
Epoch 4/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 25s 25ms/step - accuracy: 0.9818 - loss: 0.0599 - val_accuracy: 0.9894 - val_loss: 0.0346
Epoch 5/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 25s 26ms/step - accuracy: 0.9871 - loss: 0.0437 - val_accuracy: 0.9824 - val_loss: 0.0546
Epoch 6/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 25s 25ms/step - accuracy: 0.9932 - loss: 0.0224 - val_accuracy: 0.9811 - val_loss: 0.0576
Epoch 7/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 25s 25ms/step - accuracy: 0.9907 - loss: 0.0318 - val_accuracy: 0.9657 - val_loss: 0.1182
Epoch 8/20
981/981 ━━━━━━━━━━━━━━━━━━━━ 25s 26ms/step - accuracy: 0.9890 - loss: 0.0333 - 

246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9883 - loss: 0.0394
Validation Accuracy: 98.87%


In [11]:
import cv2
import numpy as np

# Class mapping dictionary
classes = { 
    1: 'Speed limit (20km/h)', 2: 'Speed limit (30km/h)', 3: 'Speed limit (50km/h)',
    4: 'Speed limit (60km/h)', 5: 'Speed limit (70km/h)', 6: 'Speed limit (80km/h)',
    7: 'End of speed limit (80km/h)', 8: 'Speed limit (100km/h)', 9: 'Speed limit (120km/h)',
    10: 'No passing', 11: 'No passing veh over 3.5 tons', 12: 'Right-of-way at intersection',
    13: 'Priority road', 14: 'Yield', 15: 'Stop', 16: 'No vehicles',
    17: 'Veh > 3.5 tons prohibited', 18: 'No entry', 19: 'General caution',
    20: 'Dangerous curve left', 21: 'Dangerous curve right', 22: 'Double curve',
    23: 'Bumpy road', 24: 'Slippery road', 25: 'Road narrows on the right',
    26: 'Road work', 27: 'Traffic signals', 28: 'Pedestrians',
    29: 'Children crossing', 30: 'Bicycles crossing', 31: 'Beware of ice/snow',
    32: 'Wild animals crossing', 33: 'End speed + passing limits', 34: 'Turn right ahead',
    35: 'Turn left ahead', 36: 'Ahead only', 37: 'Go straight or right',
    38: 'Go straight or left', 39: 'Keep right', 40: 'Keep left',
    41: 'Roundabout mandatory', 42: 'End of no passing', 43: 'End no passing veh > 3.5 tons'
}

def predict_image(image_path, model):
    # Load and preprocess the image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (32, 32))  # Resize to match training size
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Reshape for model input

    # Predict the class
    prediction = model.predict(img)
    class_id = np.argmax(prediction)  # Get the class with the highest probability

    # Get the class name
    class_name = classes.get(class_id + 1, "Unknown")  # Adjust index for correct mapping

    print(f"Predicted Class: {class_name}")
    return class_name

# Example usage
test_image_path = "D:/Traffic Signal Analysis/archive/Test/00004.png"  # Replace with actual image path
predict_image(test_image_path, model)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted Class: Right-of-way at intersection


'Right-of-way at intersection'